# Comics Rx
## [A comic book recommendation system](https://github.com/MangrobanGit/comics_rx)
<img src="https://images.unsplash.com/photo-1514329926535-7f6dbfbfb114?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=2850&q=80" width="400" align='left'>

---

# B. ALS with PySpark

---

# Libraries

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2  # 1 would be where you need to specify the files
#%aimport data_fcns

import pandas as pd # dataframes
import os
import gspread_pandas
from gspread_pandas import Spread, Client # gsheets interaction

# Data storage
from sqlalchemy import create_engine # SQL helper
import psycopg2 as psql #PostgreSQL DBs

# Custom
import data_fcns as dfc
import keys  # Custom keys lib

In [1]:
# import necessary libraries
import pyspark

In [8]:
# instantiate SparkSession object
spark = pyspark.sql.SparkSession.builder.master("local[*]").getOrCreate()
# spark = SparkSession.builder.master("local").getOrCreate()

In [9]:
from pyspark.sql.types import (StructType, StructField, IntegerType
                               ,FloatType, LongType )

In [ ]:
schema = StructType(
    [
                    StructField('userID', IntegerType())
                    ,StructField('movieID', IntegerType())
                    ,StructField('rating', FloatType())
                    ,StructField('timestamp', LongType())
    ]
)

In [ ]:
testing = spark.read.

In [ ]:
# read in the dataset into pyspark DataFrame
testing = spark.read.csv('./data/ratings.csv'
                               , inferSchema=False
                               , schema=schema
                               , header=True)